# Confidence and Choice Visualizer

Run DRAM_classify_blobs 10 times (use runDRAM_classify_blobs.sh in hydra), then copy all the checkpoints to guppy (use copyDRAM_classify_blobs_ckpts.sh in hydra and enter password 10x).

Make sure these are the <b>guppy</b> model_settings: <br />
min_edge = 2 <br />
max_edge = 5 <br />
min_blobs = 1 <br />
max_blobs = 9 (15 for DRAM_classify_blobs_2)<br />
glimpses = max_blobs + 1 <br />
batch_size = 9000 (100 on hydra)

You could then use this entire program or use ClassificationDistributions.ipynb to output confidence_one_run for each run, and set them as the confidence for each run in confidence_all_runs.

In [1]:
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
from analysis import read_n, classify_imgs2

['/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py', '-f', 'true', 'true', 'true', 'true', 'true', 'model_runs/-f/classify_log.csv', 'model_runs/-f/classifymodel_0.ckpt', 'model_runs/-f/classifymodel_', 'model_runs/-f/zzzdraw_data_5000.npy', 'false', 'true', 'false', 'false', 'true']
analysis.py


In [2]:
model_name = "DRAM_classify_blobs_4"
num_runs = 10
iter_list = [0, 250, 1000, 4000, 16000, 32000, 64000, 125000]#, 250000, 500000]#, 1000000, 2000000]
glimpse_list = [0, 3, 9]
max_blobs = 15
min_blobs = 1

num_iters = len(iter_list)
num_glimpses = len(glimpse_list)
z_size = max_blobs - min_blobs + 1
data_directory = "data/" + model_name + "/"
m = 0.5

confidence_all_runs = np.zeros([num_runs, num_iters, num_glimpses, z_size, z_size])
choice_all_runs = np.zeros([num_runs, num_iters, num_glimpses, z_size, z_size])

In [3]:
def fill_matrix(path, iteration):
    """Fill the confidence and choice matrices for one run at one iteration."""
    
    data = None
    num_imgs = 9000
    imgs_data = classify_imgs2(iteration, True, num_imgs, path=path)
        
    confidence_one_run = np.zeros([num_glimpses, z_size, z_size])
    choice_one_run = np.zeros([num_glimpses, z_size, z_size])
    
    for g, glimpse in enumerate(glimpse_list):

        for nb in range(z_size):
            num_blobs = nb + min_blobs
            confidence_hist = np.zeros(z_size)
            choice_hist = np.zeros(z_size)
            num_imgs_with_num_blobs = 0

            for idx, data in enumerate(imgs_data):

                if data["label"][nb] == 1: # data is for an image with num_blobs blobs
                    num_imgs_with_num_blobs += 1

                    # Histogram of softmaxes
                    confidence_hist += data["classifications"][glimpse][0]

                    # Histogram of choices
                    choice = np.argmax(data["classifications"][glimpse][0])
                    choice_list = [0] * z_size
                    choice_list[choice] = 1
                    choice_hist += choice_list

            confidence_hist = confidence_hist / num_imgs_with_num_blobs
            confidence_one_run[g, nb] = confidence_hist.tolist()

            choice_hist = choice_hist / num_imgs_with_num_blobs
            choice_one_run[g, nb] = choice_hist.tolist()
        
#     print("Confidence (One Run): ")
#     print(confidence_one_run.tolist())
#     print("Choice (One Run): ")
#     print(choice_one_run.tolist())
    return confidence_one_run, choice_one_run

In [4]:
for run in range(num_runs):
    path = 'model_runs/' + model_name + '_run_' + str(run + 1) # '/run_' if all the runs are stored in one folder
#     path = 'model_runs/DRAM_test_square'
    for i, iteration in enumerate(iter_list):
        confidence_all_runs[run, i], choice_all_runs[run, i] = fill_matrix(path, iteration)

INFO:tensorflow:Restoring parameters from model_runs/DRAM_classify_blobs_4_run_1/classifymodel_0.ckpt
INFO:tensorflow:Restoring parameters from model_runs/DRAM_classify_blobs_4_run_1/classifymodel_250.ckpt
INFO:tensorflow:Restoring parameters from model_runs/DRAM_classify_blobs_4_run_1/classifymodel_1000.ckpt
INFO:tensorflow:Restoring parameters from model_runs/DRAM_classify_blobs_4_run_1/classifymodel_4000.ckpt
INFO:tensorflow:Restoring parameters from model_runs/DRAM_classify_blobs_4_run_1/classifymodel_16000.ckpt
INFO:tensorflow:Restoring parameters from model_runs/DRAM_classify_blobs_4_run_1/classifymodel_32000.ckpt
INFO:tensorflow:Restoring parameters from model_runs/DRAM_classify_blobs_4_run_1/classifymodel_64000.ckpt
INFO:tensorflow:Restoring parameters from model_runs/DRAM_classify_blobs_4_run_1/classifymodel_125000.ckpt
INFO:tensorflow:Restoring parameters from model_runs/DRAM_classify_blobs_4_run_2/classifymodel_0.ckpt
INFO:tensorflow:Restoring parameters from model_runs/DRAM

INFO:tensorflow:Restoring parameters from model_runs/DRAM_classify_blobs_4_run_10/classifymodel_64000.ckpt
INFO:tensorflow:Restoring parameters from model_runs/DRAM_classify_blobs_4_run_10/classifymodel_125000.ckpt


In [5]:
def adj_all_runs(all_runs_matrix):
    """Adjust all the matrices so tick marks start with 1."""
    
    new_all_runs_matrix = np.zeros([num_runs, num_iters, num_glimpses, z_size + 1, z_size + 1])
    for m, matrix in enumerate(all_runs_matrix):
        for i in range(num_iters):
            for g in range(num_glimpses):
                new_all_runs_matrix[m, i, g] = adj_matrix(matrix[i, g])
    return new_all_runs_matrix


def adj_matrix(matrix):
    """Adjust the matrix so tick marks start with 1."""
    
    a = np.zeros([1, z_size])
    temp = np.vstack((a, matrix))
    b = np.zeros([z_size + 1, 1])
    new_matrix = np.hstack((b, temp))
    return new_matrix


confidence_all_runs_adj = adj_all_runs(confidence_all_runs)
choice_all_runs_adj = adj_all_runs(choice_all_runs)

In [6]:
confidence_avg = np.mean(confidence_all_runs_adj, axis=0)
choice_avg = np.mean(choice_all_runs_adj, axis=0)

In [7]:
def plot_confidence(iter_idx, it, g, glimpse, run=None):
    """Plot the confidence heatmap."""
    
    if run is None:
        matrix = confidence_avg[iter_idx, g]
        plot_title = "Confidence after %d Iterations at Glimpse %d" % (it, glimpse)

    else:
        matrix = confidence_all_runs_adj[run, iter_idx, g]
        plot_title = "Confidence at Run %d after %d Iterations at Glimpse %d" % (run + 1, it, glimpse)
    plot_heatmap(matrix, plot_title)


def plot_choice(iter_idx, it, g, glimpse, run=None):
    """Plot the choice heatmap."""
    
    if run is None:
        matrix = choice_avg[iter_idx, g]
        plot_title = "Choice after %d Iterations at Glimpse %d" % (it, glimpse)
    else:
        matrix = choice_all_runs_adj[run, iter_idx, g]
        plot_title = "Choice at Run %d after %d Iterations at Glimpse %d" % (run + 1, it, glimpse)
    plot_heatmap(matrix, plot_title)
    
    
def plot_heatmap(matrix, plot_title):
    """Plot heatmap."""
    
    data = [go.Heatmap(
        z=matrix,
        colorscale="Jet"
    )]

    layout = go.Layout(
        title=plot_title,
        yaxis=dict(
#             range=[max_blobs + m, min_blobs - m],
            range=[9 + m, min_blobs - m],
            title="True Class",
            dtick=1,
            tickcolor='#FFF'
        ),
        xaxis=dict(
            range=[min_blobs - m, max_blobs + m],
            title="Predicted Class",
            dtick=1,
            tickcolor='#FFF'
        ),
        width=500,
        height=500,
        plot_bgcolor="#000",
        paper_bgcolor="#000",
        font=dict(
            color="#FFF"
        ),
        titlefont=dict(
            color="#FFF"
        ),
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

In [8]:
for i, it in enumerate(iter_list):
    for g, glimpse in enumerate(glimpse_list):
        plot_confidence(i, it, g, glimpse)
        plot_choice(i, it, g, glimpse)

In [9]:
import os, errno

try:
    os.makedirs(data_directory)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [10]:
np.save(data_directory + "confidence_hist", confidence_all_runs)
np.save(data_directory + "choice_hist", choice_all_runs)

## Work with Saved Histogram Data

In [11]:
confidence_hist = np.load(data_directory + "confidence_hist.npy")
choice_hist = np.load(data_directory + "choice_hist.npy")

In [12]:
def get_msdcv(hist_matrix):
    """Get the mean, standard deviation, and coefficient of variation matrices from histogram matrix."""
    
    msdcv = np.zeros([num_runs, num_iters, num_glimpses, z_size, 3])
    
    for run in range(num_runs):
        for i, it in enumerate(iter_list):
            for g, glimpse in enumerate(glimpse_list):
                for t in range(z_size):
                    values_sum = 0
                    sqr_sum = 0

                    # Find the mean
                    for p in range(z_size):
                        values_sum += (p + 1) * hist_matrix[run, i, g, t, p]
                    msdcv[run, i, g, t, 0] = mu = values_sum

                    # Find the standard deviation
                    for p in range(z_size):
                        sqr_sum += (p + 1 - mu)**2 * hist_matrix[run, i, g, t, p]
                    msdcv[run, i, g, t, 1] = sigma = np.sqrt(sqr_sum)

                    # Find the coefficient of variation
                    msdcv[run, i, g, t, 2] = cv = sigma / mu
    return msdcv

In [13]:
confidence_msdcv = get_msdcv(confidence_hist)
choice_msdcv = get_msdcv(choice_hist)

In [14]:
np.save(data_directory + "confidence_msdcv", confidence_msdcv)
np.save(data_directory + "choice_msdcv", choice_msdcv)

## Visualize Mean, Standard Deviation, and Coefficient of Variation

In [15]:
confidence_msdcv = np.load(data_directory + "confidence_msdcv.npy")
choice_msdcv = np.load(data_directory + "choice_msdcv.npy")

In [16]:
confidence_msdcv_avg = np.mean(confidence_msdcv, axis=0)
choice_msdcv_avg = np.mean(choice_msdcv, axis=0)

def plot_confidence_msdcv(iter_idx, it, g, glimpse, run=None):
    """Plot the confidence linegraphs."""
    
    if run is None:
        matrix = confidence_msdcv_avg[iter_idx, g]
        plot_title = "Confidence after %d Iterations at Glimpse %d" % (it, glimpse)

    else:
        matrix = confidence_msdcv[run, iter_idx, g]
        plot_title = "Confidence at Run %d after %d Iterations at Glimpse %d" % (run + 1, it, glimpse)
    plot_linegraphs(matrix, plot_title)


def plot_choice_msdcv(iter_idx, it, g, glimpse, run=None):
    """Plot the choice linegraphs."""
    
    if run is None:
        matrix = choice_msdcv_avg[iter_idx, g]
        plot_title = "Choice after %d Iterations at Glimpse %d" % (it, glimpse)
    else:
        matrix = choice_msdcv[run, iter_idx, g]
        plot_title = "Choice at Run %d after %d Iterations at Glimpse %d" % (run + 1, it, glimpse)
    plot_linegraphs(matrix, plot_title)
    
    
def plot_linegraphs(matrix, plot_title):
    """Plot mean, stdev, and cv linegraphs."""
    
    plot_msd(matrix, plot_title)
    plot_cv(matrix, plot_title)

In [17]:
def plot_msd(matrix, plot_title):
    """Plot the mean and standard deviation in the same graph."""
    
    x_values = np.array(range(min_blobs, 9 + 1))
#     x_values = np.array(range(min_blobs, max_blobs + 1))
    y_mu = matrix[:, 0]
    y_sigma = matrix[:, 1]
    
    # Create traces
    mu_trace = go.Scatter(
        x = x_values,
        y = y_mu,
        name='Mean',
    )
    sigma_trace = go.Scatter(
        x = x_values,
        y = y_sigma,
        name='Standard Deviation',
        yaxis='y2'
    )

    data = [mu_trace, sigma_trace]
    
    layout = go.Layout(
        title=plot_title,
        yaxis=dict(
            range=[min_blobs - m, max_blobs + m],
            title="Mean",
            dtick=1,
            tickcolor='#FFF',
        ),
        yaxis2=dict(
            range=[0, 3],
            title='Standard Deviation',
            tickcolor='#FFF',
            overlaying='y',
            side='right',
            gridcolor='#999',
        ),
        xaxis=dict(
            range=[min_blobs - m, 9 + m],
#             range=[min_blobs - m, max_blobs + m],
            title="True Class",
            dtick=1,
            tickcolor='#FFF'
        ),
        width=500,
        height=500,
        plot_bgcolor="#000",
        paper_bgcolor="#000",
        font=dict(
            color="#FFF"
        ),
        titlefont=dict(
            color="#FFF"
        ),
        legend=dict(x=0, y=1),
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

In [18]:
def plot_cv(matrix, plot_title):
    """Plot the coefficient of variation."""
        
    x_values = np.array(range(min_blobs, 9 + 1))
#     x_values = np.array(range(min_blobs, max_blobs + 1))
    y_cv = matrix[:, 2]

    cv_trace = go.Scatter(
        x = x_values,
        y = y_cv,
        name='Coefficient of Variation'
    )
    data = [cv_trace]
    
    layout = go.Layout(
        title=plot_title,
        yaxis=dict(
            title="Coefficient of Variation",
            tickcolor='#FFF',
            showgrid=True
        ),
        xaxis=dict(
            range=[min_blobs - m, 9 + m],
#             range=[min_blobs - m, max_blobs + m],
            title="True Class",
            dtick=1,
            tickcolor='#FFF'
        ),
        width=500,
        height=300,
        plot_bgcolor="#000",
        paper_bgcolor="#000",
        font=dict(
            color="#FFF"
        ),
        titlefont=dict(
            color="#FFF"
        ),
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

In [19]:
for i, it in enumerate(iter_list):
    for g, glimpse in enumerate(glimpse_list):
        plot_confidence_msdcv(i, it, g, glimpse)
        plot_choice_msdcv(i, it, g, glimpse)